## Rule-based pipeline using SpaCy

In [1]:
# !python -m spacy download es_core_news_md

In [2]:
from utils import read_corpus, text_processor

In [3]:
corpus = read_corpus()

In [4]:
for k, v in corpus.items():
    print(f"{k}: {len(v)}")

A1: 94
A2: 62
B: 110
B1: 42


In [5]:
# TODO: preprocessor does not work on Aventura text for some reason

In [6]:
ex = corpus["B1"][4]["content"]
print(ex)
print("\n\n")
processed_ex = text_processor(ex)
print(processed_ex.text)
print("\n\n")
print(processed_ex.tokens)
print("\n\n")
print(processed_ex.lemmas)
print("\n\n")
print(processed_ex.tags)
print("\n\n")
print(processed_ex.parses)

HE tenido muchos perros y he conocido muchos más,
y puedo aseguraros que la familia de los ingratos no
existe en la raza canina.... La naturaleza ha sido
ingrata y cruel con la raza canina, dotándola de una
enfermedad horrible: la rabia.... Muchas veces, con
sólo ver á un perro con la lengua fuera y la mirada
triste, se le sacrifica bárbaramente por si rabia....
De los perros se cuentan muchas historias que parecen
inverosímiles, y son ciertas y reales como la luz del
sol. Pablo González tenía un mastín de lomo rojo, ojos
claros, y potentes colmillos; se llamaba Tony, y era
inseparable compañero de su amo en todas las excursiones
que con su jaca rabona y la escopeta colgada de
la grupa hacía desde su pueblo á los montes y dehesas
para comprar ganados.
Tony era un mastín inteligente, leal, fornido y dócil;
frotaba su enorme cabeza en las piernas de todo el
mundo; los niños del pueblo se subían á caballo sobre
su lomo, y cuando se cansaba de sufrir las impertinencias
infantiles, gruñía u